In [1]:
# Configs
dataset_dir = r'C:\Users\fardin\Projects\EnhanceSEO\datasets\extractedURLs\url_classes.csv'

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
import time
import nltk
import matplotlib.pyplot as plt

In [3]:
tokenizer = lambda doc: list(urlparse(doc))

In [4]:
df = pd.read_csv(dataset_dir)
df.dropna(inplace=True)
df.head()

,Address,Topic
0,https://en.wikipedia.org/wiki/Dunking,basketball
1,https://www.merriam-webster.com/dictionary/dunk,basketball
2,https://myviewfromthemiddleseat.com/2013/09/be...,basketball
3,https://slate.com/technology/2017/12/dunking-i...,basketball
4,https://www.reddit.com/r/Basketball/comments/y...,basketball


In [5]:
lens = []
for a in df.Address:
    lens.append(len(tokenizer(a)))

In [6]:
class_list = df.Topic.unique()
class_id = {t:i for i, t in enumerate(class_list)}
id_class = {i:t for i, t in enumerate(class_list)}
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=True)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 1024
max_char_length = 256
device

device(type='cuda')

In [8]:
class VocabFactory():
    
    def __init__(self) -> None:
        self.tokens_dict = {}
        self.lemmatizer = WordNetLemmatizer()
        self.vocab_id = {}
        self.id_vocab = {}
    
    def create_vocab(self, documents, vocab_count=10000):
        i = 0
        for doc in documents:
            if i %10000 == 0:
                print(i)
            i+=1
            tokens = list(urlparse(doc))
            for token in tokens:
                token = self.lemmatizer.lemmatize(token)
                if token not in self.tokens_dict:
                    self.tokens_dict[token] = 0
                self.tokens_dict[token] += 1
        self.finalize(vocab_count)
    
    def finalize(self, vocab_count=10000):
        all_tokens = pd.DataFrame({'tokens': list(self.tokens_dict.keys()), 'counts': list(self.tokens_dict.values())})
        all_tokens = all_tokens.sort_values(by=['counts'], ascending=False)
        tokens = all_tokens['tokens'].values[:vocab_count]
        self.vocab_id = {t: i for i, t in enumerate(tokens)}
        self.id_vocab = {i: t for i, t in enumerate(tokens)}
        
        

In [9]:
vf = VocabFactory()
vf.create_vocab(df.Address.values, vocab_count=12000)
vocab_dict = vf.vocab_id
vocab_size = len(vocab_dict)

0
10000
20000
30000


In [14]:
class CustomDataset(Dataset):
    
    def __init__(self, X, y, num_classes, dictionary, tokenizer, doc_length=256) -> None:
        super().__init__()
        self.doc_length = doc_length
        y = torch.from_numpy(np.array([class_id[c] for c in y], dtype=np.longlong))
        self.y = torch.nn.functional.one_hot(y, num_classes=num_classes).float()
        self.dictionary = dictionary
        self.tokenizer = tokenizer
        self.vocab_size = len(self.dictionary)
        
        self.X = torch.zeros((len(X), doc_length), dtype=torch.int)
        for i, doc in enumerate(X):
            tokens = self.tokenizer(doc)
            indices = torch.from_numpy(np.array([vocab_dict[t] for t in tokens if t in vocab_dict], dtype=np.longlong))
            pad_size = max(self.doc_length - len(indices), 0)
            self.X[i] = torch.nn.functional.pad(indices[:self.doc_length], (0,pad_size))
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
        
    def __len__(self):
        return len(self.y)
        

In [15]:
lemmatizer = WordNetLemmatizer()
train_dataset = CustomDataset(df_train.Address.values, df_train.Topic.values, len(class_id), vocab_dict, tokenizer, doc_length=6)
test_dataset = CustomDataset(df_test.Address.values, df_test.Topic.values, len(class_id), vocab_dict, tokenizer, doc_length=6)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True, shuffle=False)

In [16]:
for val in df_train.Address.values[:1]:
    print(train_dataset.tokenizer(val))
for val in df_train.Address.values[:2]:
    print(train_dataset.tokenizer(val))

['https', 'www.cnet.com', '/culture/first-solar-signs-massive-deal-with-china/', '', '', '']
['https', 'www.cnet.com', '/culture/first-solar-signs-massive-deal-with-china/', '', '', '']
['https', 'www.disciplemedia.com', '/building-your-community/build-a-fanbase-complete-guide/', '', '', '']


In [17]:
st = time.time()
X, y = next(iter(test_dataloader))
print(f'dataset time = {time.time() - st}')

dataset time = 0.020000457763671875


In [26]:
class CNN_for_Text(nn.Module):
    
    def __init__(self, num_embedding, batch_size, max_token_count, embedding_dim=64, dropout=0.3, num_out_features=4, *args, **kwargs) -> None:
        super(CNN_for_Text, self).__init__(*args, **kwargs)
        self.batch_size = batch_size
        self.max_token_count = max_token_count
        
        self.embedding = nn.Embedding(num_embedding, embedding_dim)
        self.conv1 = nn.Conv1d(embedding_dim, 64, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.globalpool = nn.AdaptiveMaxPool1d(64)
        self.fc1 = nn.Linear(64 * 64, 32)
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(32, num_out_features)
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.globalpool(x)
        x = F.relu(self.fc1(x.view(self.batch_size, -1)))
        x = self.dropout(x)
        x = self.fc_out(x)
        return x

In [27]:
model = CNN_for_Text(num_embedding=vocab_size, batch_size=batch_size, max_token_count=6, embedding_dim=64, num_out_features=len(class_id))

In [28]:
model(X).shape

torch.Size([1024, 12])

In [29]:
import numpy as np

In [30]:
import torchmetrics
import lightning as L

In [31]:

class ClassifierLightningModel(L.LightningModule):
    def __init__(
        self,
        model,
        num_classes,
        optimizer=None,
        loss_func=None,
        learning_rate=0.01,
        batch_size=64,
        lr_scheduler=None,
        user_lr_scheduler=False,
        min_lr=0.0,
    ):
        super(ClassifierLightningModel, self).__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model = model
        self.min_lr = min_lr
        # self.save_hyperparameters(ignore=["model"])
        self.save_hyperparameters("model", logger=False)
        self.optimizer = self._get_optimizer(optimizer)
        self.lr_scheduler = (
            self._get_lr_scheduler(lr_scheduler) if user_lr_scheduler else None
        )
        self.loss_func = loss_func
        self.train_losses = []
        self.val_losses = []
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x, *args, **kwargs):
        return self.model(x)

    def on_train_epoch_start(self) -> None:
        param_groups = next(iter(self.optimizer.param_groups))
        if "lr" in param_groups and param_groups["lr"] is not None:
            current_learning_rate = float(param_groups["lr"])
            self.log(
                "lr",
                current_learning_rate,
                batch_size=self.batch_size,
                on_epoch=True,
                on_step=False,
            )

    def training_step(self, batch, *args, **kwargs):
        X, y = batch
        X.to(self.device)
        y.to(self.device)
        
        self.model.train()
        y_out = self(X)

        loss = self.loss_func(y_out.view(y.shape), y )
        self.train_losses.append(loss.detach().item())
        self.log(
            "train_loss",
            loss,
            prog_bar=True,
            batch_size=self.batch_size,
            on_epoch=True,
            on_step=True,
        )
        
        self.train_acc(torch.argmax(y_out, dim=1), torch.argmax(y, dim=1))
        self.log('train_acc', self.train_acc, prog_bar=True, on_epoch=True, on_step=True, batch_size=self.batch_size)
        
        return loss

    def validation_step(self, batch, *args, **kwargs):
        X, y = batch
        X.to(self.device)
        y.to(self.device)
        
        self.model.eval()
        y_out = self(X)
        loss = self.loss_func(y_out.view(y.shape), y )
        self.val_losses.append(loss.detach().item())

        self.log(
            "val_loss",
            loss,
            prog_bar=True,
            batch_size=self.batch_size,
            on_epoch=True,
            on_step=False,
        )
        
        
        self.val_acc(torch.argmax(y_out, dim=1), torch.argmax(y, dim=1))
        self.log('val_acc', self.val_acc, prog_bar=True, on_epoch=True, on_step=False, batch_size=self.batch_size)

    def configure_optimizers(self):
        if self.lr_scheduler is None:
            return self.optimizer

        return {
            "optimizer": self.optimizer,
            "lr_scheduler": {
                "scheduler": self.lr_scheduler,
                "monitor": "train_loss",
                "interval": "epoch",
                "frequency": 1,
            },
        }

    def update_learning_rate(self, learning_rate: float):
        self.learning_rate = learning_rate
        for g in self.optimizer.param_groups:
            g["lr"] = learning_rate

    def _get_optimizer(self, optimizer):
        return (
            optimizer
            if optimizer is not None
            else torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        )

    def _get_lr_scheduler(self, lr_scheduler):
        return (
            lr_scheduler
            if lr_scheduler is not None
            else torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, patience=5, factor=0.5, mode="min", min_lr=self.min_lr
            )
        )


In [32]:

lr=  0.001380384264602885
# 0.00010631317724117211
output_size = 128

In [34]:
hidden_dim = 64
embedding_dim = 64
label_size = 1

classifier_torch_model = CNN_for_Text(num_embedding=vocab_size, batch_size=batch_size, max_token_count=6, num_out_features=len(class_id)).to(device)
optimizer = torch.optim.Adam(classifier_torch_model.parameters(), lr=lr, weight_decay=0.00012)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90, 120],gamma=0.5)
loss_func = torch.nn.BCEWithLogitsLoss()
classfier_lightning_model = ClassifierLightningModel(classifier_torch_model, 
                                                     num_classes=len(class_id),
                                            learning_rate=lr,
                                            batch_size=batch_size,
                                            optimizer=optimizer,
                                            loss_func=loss_func,
                                            lr_scheduler=lr_scheduler,
                                            user_lr_scheduler=True
                                            ).to(device)

In [35]:
from lightning.pytorch.loggers import CSVLogger
import lightning as L

trainer = L.Trainer(
            max_epochs=1000,
            accelerator= 'gpu' if device==torch.device("cuda") else 'cpu',
            logger=CSVLogger(save_dir='logs/', name='log2'), 
            num_sanity_val_steps=0
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [36]:
# max_epochs = 1000
# trainer.fit_loop.max_epochs = max_epochs
trainer.fit(classfier_lightning_model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)


You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | CNN_for_Text       | 932 K 
1 | loss_func | BCEWithLogitsLoss  | 0     
2 | train_acc | MulticlassAccuracy | 0     
3 | val_acc   | MulticlassAccuracy | 0     
4 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
932 K     Trainable params
0         Non-trainable params
932 K     Total params
3.730     Total estimated model params size (MB)
c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:293: The number of

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1000` reached.


In [37]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torchmetrics import ConfusionMatrix

def calculate_metrics(cl_model):
    cm = ConfusionMatrix(task="multiclass", num_classes=len(class_id))

    y_pred = []
    y_true = []

    cl_model = cl_model.eval()
    cl_model.to(device)
    for X, y in tqdm(test_dataloader):
        X = X.to(device)
        with torch.no_grad():
            y_p = cl_model(X)
            y_p = y_p.cpu()
        y_pred.append(y_p)
        y_true.append(y)
    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0)
    y_pred2 = torch.argmax(y_pred, dim=1)
    y_true2 = torch.argmax(y_true, dim=1)
    print(f'classification report: \n {classification_report(y_true2, y_pred2, digits=4)}')
    print(f'confusion matrix:\n {cm(y_pred2, y_true2)}')
    print('================================')


In [38]:
classfier_lightning_model.model = classfier_lightning_model.model.eval()
classfier_lightning_model = classfier_lightning_model.eval()
calculate_metrics(classfier_lightning_model)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 37.04it/s]

classification report: 
               precision    recall  f1-score   support

           0     0.2525    0.6731    0.3673       260
           1     0.6494    0.4689    0.5446       241
           2     0.9394    0.4960    0.6492       250
           3     0.8738    0.3782    0.5279       238
           4     0.6145    0.3764    0.4668       271
           5     0.4907    0.3050    0.3762       259
           6     0.7387    0.3037    0.4304       270
           7     0.8586    0.3014    0.4462       282
           8     0.4447    0.7979    0.5711       287
           9     0.4323    0.5415    0.4808       277
          10     0.3701    0.6314    0.4667       255
          11     0.7794    0.5824    0.6667       182

    accuracy                         0.4870      3072
   macro avg     0.6203    0.4880    0.4995      3072
weighted avg     0.6132    0.4870    0.4942      3072

confusion matrix:
 tensor([[175,  14,   2,   0,   8,   1,   4,   3,  33,   8,   8,   4],
        [ 89, 113, 